# CNN to detect Football event from non football event

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
from torchvision import transforms, models, datasets
import numpy as np
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from PIL import Image
Image.LOAD_TRUNCATED_IMAGES = True
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm.notebook import tqdm
import os
import zipfile
import timm

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [ ]:
model = timm.create_model('dm_nfnet_f0.dm_in1k', pretrained=True).to(DEVICE)

In [ ]:
# torch.save(model.state_dict(), 'dm_nfnet.pth')

In [ ]:
model.get_classifier()


Linear(in_features=3072, out_features=1000, bias=True)

In [ ]:
model.get_classifier().out_features = 2

In [ ]:
model.get_classifier()

Linear(in_features=3072, out_features=2, bias=True)

In [ ]:
# Freezing all the layers except the Classifier for the transfer learning
for param in model.parameters():
    param.requires_grad = False

for param in model.get_classifier().parameters():
    param.requires_grad = True


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [ ]:
def save_checkpoint(model, optimizer, train_loss, test_loss, train_acc, valid_acc, epoch, batch_idx, checkpoint_path):
    state = {'model': model.state_dict(),
             'optimizer': optimizer.state_dict(),
             'train_loss': train_loss,
             'test_loss': test_loss,
             'train_acc': train_acc,
             'valid_acc': valid_acc,
             'epoch': epoch,
             'batch_idx': batch_idx}
    torch.save(state, checkpoint_path)
    print('Model saved to %s.' % checkpoint_path)

In [ ]:
def train(model, criterion, optimizer, train_loader, val_loader, save_location, checkpoint_location, early_stop=3, n_epochs=10, print_every=1):
    # Initializing variables
    valid_acc_max = 0
    stop_count = 0
    model.epochs = 0

    train_acc_list = []
    val_acc_list = []

    train_loss_list = []
    val_loss_list = []

    model.to(DEVICE)

    # Loop starts here
    for epoch in tqdm(range(n_epochs)):
        train_loss = 0
        valid_loss = 0

        train_acc = 0
        valid_acc = 0

        model.train()

        # Batch control
        batch_num = 0

        for data, label in train_loader:
            batch_num += 1
            data = data.to(DEVICE)  # Move data to the same device as the model
            label = label.to(DEVICE)  # Move label to the same device as the model

            output = model(data)

            loss = criterion(output, label)
            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)

            correct_tensor = pred.eq(label.data.view_as(pred))
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            train_acc += accuracy.item() * data.size(0)

            if batch_num % 10 == 0:
                print(f'Epoch: {epoch}\t{100 * (batch_num + 1) / len(train_loader):.2f}% complete.')

        model.epochs += 1

        if val_loader is not None:
            with torch.no_grad():
                model.eval()
                for data, label in val_loader:
                    data = data.to(DEVICE)  # Move data to the same device as the model
                    label = label.to(DEVICE)  # Move label to the same device as the model

                    output = model(data)
                    loss = criterion(output, label)
                    valid_loss += loss.item() * data.size(0)

                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(label.data.view_as(pred))
                    accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
                    valid_acc += accuracy.item() * data.size(0)

            valid_loss = valid_loss / len(val_loader.dataset)
            valid_acc = valid_acc / len(val_loader.dataset)

        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_acc / len(train_loader.dataset)

        train_acc_list.append(train_acc)
        train_loss_list.append(train_loss)
        val_acc_list.append(valid_acc)
        val_loss_list.append(valid_loss)

        save_checkpoint(model, optimizer, train_loss_list, val_loss_list, train_acc_list, val_acc_list, epoch, batch_num,
                        checkpoint_location)

        if (epoch + 1) % print_every == 0:
            print(f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}')
            print(f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%')

            if valid_acc > valid_acc_max:
                torch.save({'state_dict': model.state_dict()}, save_location)
                stop_count = 0
                valid_acc_max = valid_acc
                best_epoch = epoch
            else:
                stop_count += 1
                if stop_count >= early_stop:
                    print(f'\nEarly Stopping Total epochs: {epoch}. Best epoch: {best_epoch} with best val acc: {100 * valid_acc_max:.2f}%')
                    model.load_state_dict(torch.load(save_location)['state_dict'])
                    model.optimizer = optimizer
                    return model

    model.optimizer = optimizer
    return model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if False:
  zip_ref = zipfile.ZipFile('/content/drive/MyDrive/trial.zip', 'r')
  zip_ref.extractall()

In [ ]:
# Creating the dataset
data_dir = 'trial'
event_dir = ['Cards', 'Corner', 'Event', 'Free-Kick', 'Penalty', 'Red-Cards', 'Tackle', 'To_Subtitue', 'Yellow-Cards']
non_event_dir = ['Center', 'Left', 'Right']

In [ ]:


# The data that is in ./trial/event is class 0 and ./trial/non_event is class 1
# The data is split into 80% train and 20% test
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, event_dir, non_event_dir, transform=None):
        self.data_dir = data_dir
        self.event_dir = event_dir
        self.non_event_dir = non_event_dir
        self.transform = transform
        self.data = []
        self.label = []
        self.event_len = 0
        self.non_event_len = 0

        for event in self.event_dir:
            event_path = os.path.join(self.data_dir, event)
            for file in os.listdir(event_path):
                self.data.append(os.path.join(event_path, file))
                self.label.append(0)
                self.event_len += 1

        for non_event in self.non_event_dir:
            non_event_path = os.path.join(self.data_dir, non_event)
            for file in os.listdir(non_event_path):
                self.data.append(os.path.join(non_event_path, file))
                self.label.append(1)
                self.non_event_len += 1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = Image.open(self.data[idx])
        img = img.convert('RGB')
        label = self.label[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

    def get_event_len(self):
        return self.event_len

    def get_non_event_len(self):
        return self.non_event_len

    def get_data(self):
        return self.data

    def get_label(self):
        return self.label





In [ ]:
thedata = CustomDataset(data_dir, event_dir, non_event_dir, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]))

In [ ]:
# Splitting the data into train and test
TRAIN_PCT = 0.8
train_size = int(TRAIN_PCT * len(thedata))
test_size = len(thedata) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(thedata, [train_size, test_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
model = train(model, criterion, optimizer, train_loader, test_loader, save_location='nfnet_best.pth.tar',checkpoint_location='nfnet_checkpoint.pth.tar')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0	1.42% complete.
Epoch: 0	2.72% complete.
Epoch: 0	4.02% complete.
Epoch: 0	5.31% complete.
Epoch: 0	6.61% complete.
Epoch: 0	7.90% complete.
Epoch: 0	9.20% complete.
Epoch: 0	10.49% complete.
Epoch: 0	11.79% complete.
Epoch: 0	13.08% complete.
Epoch: 0	14.38% complete.
Epoch: 0	15.67% complete.
Epoch: 0	16.97% complete.
Epoch: 0	18.26% complete.
Epoch: 0	19.56% complete.
Epoch: 0	20.85% complete.
Epoch: 0	22.15% complete.
Epoch: 0	23.45% complete.
Epoch: 0	24.74% complete.
Epoch: 0	26.04% complete.
Epoch: 0	27.33% complete.
Epoch: 0	28.63% complete.
Epoch: 0	29.92% complete.
Epoch: 0	31.22% complete.
Epoch: 0	32.51% complete.
Epoch: 0	33.81% complete.
Epoch: 0	35.10% complete.
Epoch: 0	36.40% complete.
Epoch: 0	37.69% complete.
Epoch: 0	38.99% complete.
Epoch: 0	40.28% complete.
Epoch: 0	41.58% complete.
Epoch: 0	42.88% complete.
Epoch: 0	44.17% complete.
Epoch: 0	45.47% complete.
Epoch: 0	46.76% complete.
Epoch: 0	48.06% complete.
Epoch: 0	49.35% complete.
Epoch: 0	50.65% com